# Geology and Production Data Munging

First we will import pandas so we can read in the data and clean it up. The end goal is that we want a `DataFrame` with some features related to completions, geology, location, and other variables. We ultimately want to predict the first 18 months of oil production from the horizontal wells, so that will have to be in the `DataFrame` as well

In [1]:
import pandas as pd

We first read in the excel workbook using `pandas` `pd.ExcelFile` and then print out a list of the different sheet names in the workbook using `sheet_names`

In [2]:
xls = pd.ExcelFile(r'Appendix1_DataTables.xlsx')
xls.sheet_names #list out the names of the different sheets in the excel file

['Explanation',
 'HorizontalWellCompletion',
 'UnconventionalProduction',
 'ConventionalProduction',
 'APIgravity',
 'GasComposition',
 'GasOilRatio',
 'Pressure',
 'Temperature']

From the `sheet_names` call we see that there are sheets for completions, production, oil gravity, composition, gas oil ratios, pressure and temperature. Let's use `parse` to make a `dataframe` for each sheet

In [3]:
completions = xls.parse(1) #make a dataframe from the completions sheet
production = xls.parse(2) #make a dataframe from the productions sheet
gravity = xls.parse(4) #make a dataframe from the API gravity sheet
composition = xls.parse(5, header=1) #make a dataframe from the gas composition sheet
gor = xls.parse(6) #make a dataframe from the gas oil ratio sheet
pressure = xls.parse(7) #make a dataframe from the pressure sheet
temperature = xls.parse(8) #make a dataframe from the temperature sheet

Now let's select which columns we want from each dataframe. Here we are going to select the `'APINO'` which is the API number or unique identifier for each well, and some features. We want to select the API number so we can merge everything into one dataframe. From the production dataframe we want `'First 18 months oil (bbl)'` which is the feature we are going to try and predict from the rest of the features. Other features we are selecting include oil gravity, well surface location, well bottom location, well type, reservoir name, gas oil ratio, pressure gradient, and reservoir temperature. 

We are making subsets of the original dataframes in this next cell and then combining them into a list. 

In [4]:
production = production[['APINO', 'First 18 months oil (bbl)']].copy() #this is the feature we are going to predict
gravity = gravity[['APINO', 'IP oil API gravity (°)']].copy() #oil gravity feature
composition = composition.drop(['API number', 'Surface hole latitude (NAD83)', 'Surface hole longitude (NAD83)', 'Well type', 'WSGS reservoir'], axis=1).copy()
#composition is the geochemistry of the oil, it's easier to just drop features for this one than select all the features
pressure = pressure[['APINO', 'Pressure gradient (psi/ft)']] #pressure gradient for each well
temperature = temperature[['APINO', 'Calculated top of reservoir temperature (°F)']].copy() #top of reservoir temp feature

The gas oil ratio `dataframe` has some issues with mixed datatypes in that we have numbers for some rows and strings for other rows. We will clean this up in the next cell and make sure the `'Average gas-oil ratio (ft3/bbl)'` feature is composed of numbers and not mixed datatypes

In [5]:
gor = gor[['APINO', 'Average gas-oil ratio (ft3/bbl)']].copy()
gor2 = gor[~gor['Average gas-oil ratio (ft3/bbl)'].isin(['*IHS proprietary data'])].copy() #this one needed some thorough cleaning because of mixed data types
gor2['Average gas-oil ratio (ft3/bbl)'] = pd.to_numeric(gor2['Average gas-oil ratio (ft3/bbl)']) #make sure all entries are numbers

In [6]:
dataframes = [completions, production, gravity, gor2, composition, temperature, pressure] #this is a list of all the dataframes with the features we want to use

Now let's merge the dataframes together using the list from the cell above. We do this using the reduce function https://docs.python.org/2/library/functions.html#reduce

In [7]:
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right, how='left', on='APINO'), dataframes)
df_final.head() #take a quick look to see what the data looks like

,API number,APINO,Company,Surface hole latitude (NAD83),Surface hole longitude (NAD83),Bottom hole latitude (NAD83),Bottom hole longitude (NAD83),Well type,WSGS reservoir,Total vertical depth at bottom hole (ft),...,C2,C3,iC4,nC4,iC5,nC5,dryness_ratio C1/(ΣC1-C5),iC4/nC4,Calculated top of reservoir temperature (°F),Pressure gradient (psi/ft)
0,49-005-43307,543307,THE TERMO COMPANY,44.06884,-105.13920,44.07319,-105.122710,H,Turner,6325.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159.018300,NaN
1,49-005-45589,545589,EOG Y RESOURCES INC,43.67997,-105.66802,43.69113,-105.672250,H,Turner,10689.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.851380,NaN
2,49-005-60281,560281,EOG RESOURCES INC,43.53329,-105.43121,43.52093,-105.427393,H,Turner,10114.29,...,12.737,6.322,1.051,1.825,0.798,0.633,0.743112,0.57589,239.576407,NaN
3,49-005-60880,560880,EOG RESOURCES INC,43.53269,-105.46218,43.52074,-105.455890,H,Turner,10118.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,251.653471,NaN
4,49-005-60883,560883,EOG RESOURCES INC,43.53544,-105.42470,43.54680,-105.433520,H,Turner,10025.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,246.216390,NaN


At first glance I see a lot of NaN values in the dataset, we don't have a complete dataset for the gas composition. This means we only have a few wells with that data. We can either impute the data using a mean or median imputation, or we can drop some of the features. To decide which features to drop we will set a threshold and if a feature is missing more than that threshold we will drop that feature from the dataframe. Here I use a 25% threshold to drop features (columns) missing more than 25% of their data.

In [8]:
thresh = len(df_final) * .25
df_final.dropna(thresh = thresh, axis = 1, inplace = True) #axis 1 means drop columns, inplace means to modify the dataframe

In [9]:
df_final.columns #just a quick sanity check to see what features we still have left in the dataframe

Index(['API number', 'APINO', 'Company', 'Surface hole latitude (NAD83)',
       'Surface hole longitude (NAD83)', 'Bottom hole latitude (NAD83)',
       'Bottom hole longitude (NAD83)', 'Well type', 'WSGS reservoir',
       'Total vertical depth at bottom hole (ft)', 'Number of frac stages',
       'Total slurry (bbl)', 'Total proppant (lb)',
       'Surface-to-bottom hole length (ft)', 'Producing interval length (ft)',
       'Lateral azimuth (°)', 'First 18 months oil (bbl)',
       'IP oil API gravity (°)', 'Average gas-oil ratio (ft3/bbl)',
       'Calculated top of reservoir temperature (°F)'],
      dtype='object')

After thresholding and dropping some features we still have `NaN` values, and we can impute those values using the `mean` for each feature

In [10]:
impute_df = df_final.fillna(df_final.mean())
impute_df.head() #another check to see what the dataframe looks like

,API number,APINO,Company,Surface hole latitude (NAD83),Surface hole longitude (NAD83),Bottom hole latitude (NAD83),Bottom hole longitude (NAD83),Well type,WSGS reservoir,Total vertical depth at bottom hole (ft),Number of frac stages,Total slurry (bbl),Total proppant (lb),Surface-to-bottom hole length (ft),Producing interval length (ft),Lateral azimuth (°),First 18 months oil (bbl),IP oil API gravity (°),Average gas-oil ratio (ft3/bbl),Calculated top of reservoir temperature (°F)
0,49-005-43307,543307,THE TERMO COMPANY,44.06884,-105.13920,44.07319,-105.122710,H,Turner,6325.00,21.275862,85183.453893,4.967583e+06,4613.209075,3831.0,75.222237,776.0,42.653571,3793.832829,159.018300
1,49-005-45589,545589,EOG Y RESOURCES INC,43.67997,-105.66802,43.69113,-105.672250,H,Turner,10689.62,21.275862,85183.453893,1.251240e+05,4217.598301,3533.0,339.241676,57154.0,42.653571,3793.832829,199.851380
2,49-005-60281,560281,EOG RESOURCES INC,43.53329,-105.43121,43.52093,-105.427393,H,Turner,10114.29,12.000000,85183.453893,1.992847e+06,4615.909771,3754.0,162.838330,75721.0,52.200000,7480.932203,239.576407
3,49-005-60880,560880,EOG RESOURCES INC,43.53269,-105.46218,43.52074,-105.455890,H,Turner,10118.80,13.000000,36638.000000,1.992421e+06,4662.652084,4000.0,152.239483,68301.0,42.653571,10961.748634,251.653471
4,49-005-60883,560883,EOG RESOURCES INC,43.53544,-105.42470,43.54680,-105.433520,H,Turner,10025.20,13.000000,85183.453893,2.027081e+06,4753.829417,4072.0,322.173933,52078.0,42.653571,9099.137931,246.216390


How are we going to deal with the non-numeric features like `'Company'` and `'WSGS reservoir'`? We need to convert them to a numeric label using a label encoder. We will use `preprocessing.LabelEncoder` to convert categorical entries to numerical values. So to do this let's import the `preprocessing` module and fit the `LabelEncoder` to the categorical type data. We will then join the numeric `dataframe` and the categorical `dataframe` into one encoded `dataframe`

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

X_numeric = impute_df.select_dtypes(include=['number']).copy()
X_categorical = impute_df.select_dtypes(include=['object']).apply(le.fit_transform)
df_encoded = X_numeric.join( X_categorical, how='outer')

And here we have our cleaned, organized, imputed `DataFrame` with all our features of interest ready for mahcine learning. Next week, we will dive into using scikit-learn to determine which features are the most important when predicting the first 18 months of oil production in the Turner Sandstone

In [12]:
df_encoded.head()

,APINO,Surface hole latitude (NAD83),Surface hole longitude (NAD83),Bottom hole latitude (NAD83),Bottom hole longitude (NAD83),Total vertical depth at bottom hole (ft),Number of frac stages,Total slurry (bbl),Total proppant (lb),Surface-to-bottom hole length (ft),Producing interval length (ft),Lateral azimuth (°),First 18 months oil (bbl),IP oil API gravity (°),Average gas-oil ratio (ft3/bbl),Calculated top of reservoir temperature (°F),API number,Company,Well type,WSGS reservoir
0,543307,44.06884,-105.13920,44.07319,-105.122710,6325.00,21.275862,85183.453893,4.967583e+06,4613.209075,3831.0,75.222237,776.0,42.653571,3793.832829,159.018300,0,22,1,1
1,545589,43.67997,-105.66802,43.69113,-105.672250,10689.62,21.275862,85183.453893,1.251240e+05,4217.598301,3533.0,339.241676,57154.0,42.653571,3793.832829,199.851380,1,9,1,1
2,560281,43.53329,-105.43121,43.52093,-105.427393,10114.29,12.000000,85183.453893,1.992847e+06,4615.909771,3754.0,162.838330,75721.0,52.200000,7480.932203,239.576407,2,8,1,1
3,560880,43.53269,-105.46218,43.52074,-105.455890,10118.80,13.000000,36638.000000,1.992421e+06,4662.652084,4000.0,152.239483,68301.0,42.653571,10961.748634,251.653471,3,8,1,1
4,560883,43.53544,-105.42470,43.54680,-105.433520,10025.20,13.000000,85183.453893,2.027081e+06,4753.829417,4072.0,322.173933,52078.0,42.653571,9099.137931,246.216390,4,8,1,1


Let's save our munged data to a `.csv` file so we can read it in next week to make predictions

In [13]:
df_encoded.to_csv('organized_turner_data.csv')

This notebook is licensed as CC-BY, use and share to your hearts content.